# This notebook contains the full code described in the blog post about data munging with the Pitchfork album review dataset:
## https://methodmatters.blogspot.com/2019/07/showing-some-respect-for-data-munging.html 

## The data are available on Kaggle at this link:
### https://www.kaggle.com/nolanbconaway/pitchfork-data/

#### The over-arching goal of this notebook is to make a tidy dataset for analysis. We are interested in the data at the album level. This means that we need to have 1 row in our dataset for each album, with information about the albums contained in the columns.

#### Note that the Pitchfork album data on the Kaggle website are contained in an sqlite database, which contains 6 different tables. This script extracts the tables and munges them one-by-one, before merging all of the cleaned data together to create the final tidy dataset.

#### Comments and feedback are welcome!

# Data Munging Preparation: Extract the data from sqlite

In [1]:
# import needed libraries
import sqlite3, datetime
import pandas as pd
import numpy as np

In [2]:
# define directory where the data sit
in_dir = 'D:\\Directory\\'

In [3]:
# What are the tables in the database?
con = sqlite3.connect(in_dir + 'database.sqlite')
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", con)

,name
0,reviews
1,artists
2,genres
3,labels
4,years
5,content


In [4]:
# extract the tables from sql to Pandas dataframes
reviews = pd.read_sql('SELECT * FROM reviews', con)
artists = pd.read_sql('SELECT * FROM artists', con)
genres = pd.read_sql('SELECT * FROM genres', con)
labels = pd.read_sql('SELECT * FROM labels', con)
years = pd.read_sql('SELECT * FROM years', con)
content = pd.read_sql('SELECT * FROM content', con)
# and close the connection to the sql database
con.close()

In [5]:
# print out the heads of all of the tables
table_names = ['reviews', 'artists', 'genres', 'labels', 'years', 'content']
for i in np.arange(0, len(table_names)):
    print(table_names[i] + ': ')
    print((eval(table_names[i])).head())

reviews: 
   reviewid                 title            artist  \
0     22703             mezzanine    massive attack   
1     22721          prelapsarian          krallice   
2     22659  all of them naturals      uranium club   
3     22661           first songs  kleenex, liliput   
4     22725             new start              taso   

                                                 url  score  best_new_music  \
0  http://pitchfork.com/reviews/albums/22703-mezz...    9.3               0   
1  http://pitchfork.com/reviews/albums/22721-prel...    7.9               0   
2  http://pitchfork.com/reviews/albums/22659-all-...    7.3               0   
3  http://pitchfork.com/reviews/albums/22661-firs...    9.0               1   
4  http://pitchfork.com/reviews/albums/22725-new-...    8.1               0   

           author               author_type    pub_date  pub_weekday  pub_day  \
0     nate patrin               contributor  2017-01-08            6        8   
1        zoe camp     

# Part 1: Munging Reviews Data

In [6]:
# what does it look like?
reviews.head()

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017
3,22661,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017
4,22725,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017


In [7]:
# there are four duplicated review id's
reviews.reviewid.duplicated().sum()

4

In [8]:
# which review ids are they?
reviews.reviewid[reviews.reviewid.duplicated()]

12120    9417
12121    9505
12122    9499
12123    9460
Name: reviewid, dtype: int64

In [9]:
# look at them
reviews[reviews.reviewid == 9417]

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
12116,9417,radiodread,easy star all-stars,http://pitchfork.com/reviews/albums/9417-radio...,7.0,0,joe tangari,contributor,2006-10-11,2,11,10,2006
12120,9417,radiodread,easy star all-stars,http://pitchfork.com/reviews/albums/9417-radio...,7.0,0,joe tangari,contributor,2006-10-11,2,11,10,2006


In [10]:
# look at them
reviews[reviews.reviewid == 9505]

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
12117,9505,the dfa remixes: chapter two,various artists,http://pitchfork.com/reviews/albums/9505-the-d...,8.2,0,tim finney,None,2006-10-10,1,10,10,2006
12121,9505,the dfa remixes: chapter two,various artists,http://pitchfork.com/reviews/albums/9505-the-d...,8.2,0,tim finney,None,2006-10-10,1,10,10,2006


In [11]:
# look at them
reviews[reviews.reviewid == 9499]

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
12118,9499,young machetes,the blood brothers,http://pitchfork.com/reviews/albums/9499-young...,6.2,0,jason crock,None,2006-10-10,1,10,10,2006
12122,9499,young machetes,the blood brothers,http://pitchfork.com/reviews/albums/9499-young...,6.2,0,jason crock,None,2006-10-10,1,10,10,2006


In [12]:
# look at them
reviews[reviews.reviewid == 9460]

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
12119,9460,subliminal genocide,xasthur,http://pitchfork.com/reviews/albums/9460-subli...,7.8,0,brandon stosuy,contributor,2006-10-10,1,10,10,2006
12123,9460,subliminal genocide,xasthur,http://pitchfork.com/reviews/albums/9460-subli...,7.8,0,brandon stosuy,contributor,2006-10-10,1,10,10,2006


In [13]:
# conclusion- there are 4 duplicated reviewid's in the reviews database
# but as the lines are the same- we can simply drop duplicates
# we also reset the index, also
reviews_final = reviews.drop_duplicates().reset_index(drop = True)
# check that we've removed them
# looks ok!
print(reviews.shape)
print(reviews_final.shape)

(18393, 13)
(18389, 13)


# Part 2: Munging Content Data

In [14]:
# what does it look like?
content.head()

,reviewid,content
0,22703,"“Trip-hop” eventually became a ’90s punchline,..."
1,22721,"Eight years, five albums, and two EPs in, the ..."
2,22659,Minneapolis’ Uranium Club seem to revel in bei...
3,22661,Kleenex began with a crash. It transpired one ...
4,22725,It is impossible to consider a given release b...


In [15]:
# also 4 duplicates here
content.reviewid.duplicated().sum()

4

In [16]:
# here are the duplicated review id's
# these are the same ones as in the reviews table above 
content.reviewid[content.reviewid.duplicated()]

12120    9417
12121    9505
12122    9499
12123    9460
Name: reviewid, dtype: int64

In [17]:
# look at them
content[content.reviewid == 9417]

,reviewid,content
12116,9417,\r\n A song-for-song reggae cover of Radioh...
12120,9417,\r\n A song-for-song reggae cover of Radioh...


In [18]:
# look at them
content[content.reviewid == 9505]

,reviewid,content
12117,9505,\nOn the one hand it is a largely superfluous ...
12121,9505,\nOn the one hand it is a largely superfluous ...


In [19]:
# look at them
content[content.reviewid == 9499]

,reviewid,content
12118,9499,"When we last left our heroes, the Blood Brothe..."
12122,9499,"When we last left our heroes, the Blood Brothe..."


In [20]:
# look at them
content[content.reviewid == 9460]

,reviewid,content
12119,9460,Strange things are a foot in the bowels of hel...
12123,9460,Strange things are a foot in the bowels of hel...


In [21]:
# conlusion- we can simply drop the duplicates; the texts are the same
content_final = content.drop_duplicates().reset_index(drop = True)
# check that we have dropped the rows
# looks ok!|
print(content.shape)
print(content_final.shape)

(18393, 2)
(18389, 2)


# Part 3: Munging Years Data

In [22]:
years.head()

,reviewid,year
0,22703,1998.0
1,22721,2016.0
2,22659,2016.0
3,22661,2016.0
4,22725,2016.0


In [23]:
reviews[reviews.reviewid == 1535]

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
18388,1535,let us replay!,coldcut,http://pitchfork.com/reviews/albums/1535-let-u...,8.9,0,james p. wisdom,None,1999-01-26,1,26,1,1999


In [24]:
# 406 albums don't have a year...
years.isnull().sum()

reviewid      0
year        406
dtype: int64

In [25]:
# are there duplicate review ids?
# yes a lot (719)
years.reviewid.duplicated().sum()

719

In [26]:
# from the data documentation: 
# "Likewise, Pitchfork often reviews reissues and lists the original release 
# year along with the reissue year. So it's not always safe to assume that the 
# release year is the same as the publication year." 

In [27]:
# https://stackoverflow.com/questions/22798934/pandas-long-to-wide-reshape
# https://stackoverflow.com/questions/42310781/pandas-long-to-wide
# we transform the data from long to wide, putting the years in the columns
years['idx'] = years.groupby('reviewid').cumcount()
years_final = years.pivot_table("year", "reviewid", "idx", aggfunc=lambda x: max(x))
# we have a maximum of 2 years per album in these data
# rename the columns
years_final.columns = ['year1','year2']
years_final.reset_index(level=0, inplace=True)

In [28]:
# check it out
years_final.head()

,reviewid,year1,year2
0,1,2004.0,NaN
1,6,2003.0,NaN
2,7,2002.0,NaN
3,8,2005.0,NaN
4,10,2002.0,NaN


# Part 4: Munging Genre Data

In [29]:
genres.head()

,reviewid,genre
0,22703,electronic
1,22721,metal
2,22659,rock
3,22661,rock
4,22725,electronic


In [30]:
# many duplicated review id's
genres.reviewid.duplicated().sum()

4291

In [31]:
# what are some of the review ids?
genres.reviewid[genres.reviewid.duplicated()][0:5]

18    22719
21    22691
26    22695
37    22469
42    22685
Name: reviewid, dtype: int64

In [32]:
# why does this happen? Let's look at a couple of examples
# it appears that a given album can have multiple genres. makes sense
genres[genres.reviewid == 22719]

,reviewid,genre
17,22719,rap
18,22719,pop/r&b


In [33]:
# this one has three, for example
genres[genres.reviewid == 8005]

,reviewid,genre
22607,8005,jazz
22608,8005,pop/r&b
22609,8005,electronic


In [34]:
# make pandas dummy variables out of the genres
# still multiple rows per review id here
dummy_genres = pd.get_dummies(genres.genre)
# the number of rows is still the same as the genres table
print(dummy_genres.shape)
print(genres.shape)

(22680, 9)
(22680, 2)


In [35]:
# what does it look like?
dummy_genres.head()

,electronic,experimental,folk/country,global,jazz,metal,pop/r&b,rap,rock
0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0


In [36]:
# merge the dummies back into the genre database
genres_wdummies = pd.concat([genres,dummy_genres], axis = 1)
print(genres_wdummies.shape)

(22680, 11)


In [37]:
# aggregate to the reviewid level; we take the sum of the dummies
# each dummy should only exist once
# we essentially get boolean indices for each reviewid 
# of the genres represented by the album for a given reviewid
genres_wdummies_gb = genres_wdummies.groupby('reviewid').sum()
print(genres_wdummies_gb.shape)

(18389, 9)


In [38]:
genres_wdummies_gb.head()

,electronic,experimental,folk/country,global,jazz,metal,pop/r&b,rap,rock
reviewid,,,,,,,,,
1,0,0,0,0,0,0,0,0,1
6,1,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0
10,0,0,0,0,1,0,0,0,0


In [39]:
# some have more than 1 entry per genre
genres_wdummies_gb.max()

electronic      1
experimental    1
folk/country    1
global          2
jazz            1
metal           2
pop/r&b         1
rap             1
rock            2
dtype: uint8

In [40]:
# one review with global twice
# we recognize this review id!
# it's one of the ones that was repeated above
genres_wdummies_gb[genres_wdummies_gb['global'] > 1]

,electronic,experimental,folk/country,global,jazz,metal,pop/r&b,rap,rock
reviewid,,,,,,,,,
9417,0,0,0,2,0,0,0,0,0


In [41]:
# what does it look like in the original data?
genres[genres.reviewid == 9417]

,reviewid,genre
14626,9417,global
14631,9417,global


In [42]:
# let's look at the review with metal and rock twice
genres_wdummies_gb[genres_wdummies_gb['metal'] > 1]

,electronic,experimental,folk/country,global,jazz,metal,pop/r&b,rap,rock
reviewid,,,,,,,,,
9460,0,0,0,0,0,2,0,0,2


In [43]:
# we recognize this review id!
# it's one of the ones that was repeated above
genres[genres.reviewid == 9460]

,reviewid,genre
14629,9460,rock
14630,9460,metal
14634,9460,rock
14635,9460,metal


In [44]:
# conclusion: we can just binarize everything - if it's 2, it should be 1
# binarize the dataframe: set all values to 0 or 1
genres_final = genres_wdummies_gb.apply(lambda x: np.where(x == 0, 0, 1), axis = 0)
print(genres_final.shape)
genres_final.reset_index(level=0, inplace=True)
print(genres_final.shape)

(18389, 9)
(18389, 10)


In [45]:
# now we're good!
genres_final.max()

reviewid        22745
electronic          1
experimental        1
folk/country        1
global              1
jazz                1
metal               1
pop/r&b             1
rap                 1
rock                1
dtype: int64

In [46]:
genres_final.head()

,reviewid,electronic,experimental,folk/country,global,jazz,metal,pop/r&b,rap,rock
0,1,0,0,0,0,0,0,0,0,1
1,6,1,0,0,0,0,0,0,0,0
2,7,1,0,0,0,0,0,0,0,0
3,8,1,0,0,0,0,0,0,0,0
4,10,0,0,0,0,1,0,0,0,0


# Part 5: Munging the Labels

In [47]:
labels.head()

,reviewid,label
0,22703,virgin
1,22721,hathenter
2,22659,static shock
3,22659,fashionable idiots
4,22661,kill rock stars


In [48]:
# many duplicated review id's
labels.reviewid.duplicated().sum()


1801

In [49]:
# here are the duplicated review id's
labels.reviewid[labels.reviewid.duplicated()][0:5]

3     22659
5     22661
16    22720
37    22469
44    22689
Name: reviewid, dtype: int64

In [50]:
# let's check out an example
# some albums are released on multiple labels
# makes sense
labels[labels.reviewid == 22659]


,reviewid,label
2,22659,static shock
3,22659,fashionable idiots


In [51]:
# same story here - multiple labels
# for a single album
labels[labels.reviewid == 22661]

,reviewid,label
4,22661,kill rock stars
5,22661,mississippi


In [52]:
# how many labels are there?
# quite a lot
len(labels.label.unique())

3668

In [53]:
# what to do?
# we could make dummies - but that gives us 3668 extra columns
# my sense is that this isn't something that will be used very much
# we can always reverse engineer this if we have to

# here, we just do a pivot table and just stick them in a couple of columns
# does not explode the columns of our base table

In [54]:
# https://stackoverflow.com/questions/22798934/pandas-long-to-wide-reshape
# we use the same procedure as above to put the labels in the columns
labels['idx'] = labels.groupby('reviewid').cumcount()
labels_final = labels.pivot(index='reviewid', columns='idx', values='label')
labels_final.columns = ['label1','label2','label3','label4','label5']
labels_final.reset_index(level=0, inplace=True)
print(labels_final.shape)

(18389, 6)


In [55]:
# what does it look like?
labels_final.head()

,reviewid,label1,label2,label3,label4,label5
0,1,rough trade,NaN,NaN,NaN,NaN
1,6,silber,NaN,NaN,NaN,NaN
2,7,darla,NaN,NaN,NaN,NaN
3,8,darla,NaN,NaN,NaN,NaN
4,10,troubleman unlimited,NaN,NaN,NaN,NaN


# Part 6: Munging the Artists

In [56]:
# what does it look like?
artists.head()

,reviewid,artist
0,22703,massive attack
1,22721,krallice
2,22659,uranium club
3,22661,kleenex
4,22661,liliput


In [57]:
# many duplicated review id's
artists.reviewid.duplicated().sum()

442

In [58]:
# here are some of the duplicated review id's
artists.reviewid[artists.reviewid.duplicated()][0:5]

4     22661
16    22699
20    22719
51    22640
72    22672
Name: reviewid, dtype: int64

In [59]:
# why does this happen?
# of course, because a given album can have multiple artists!
artists[artists.reviewid == 22661]

,reviewid,artist
3,22661,kleenex
4,22661,liliput


In [60]:
# check out another example
# makes sense
artists[artists.reviewid == 22699]

,reviewid,artist
15,22699,smoke dza
16,22699,pete rock


In [61]:
# how many artists are there?
# quite a lot
len(artists.artist.unique())

8633

In [62]:
# so what to do?
# we could make dummies - but that gives us 8633 extra columns
# my sense is that this isn't something that will be used very much
# we can always reverse engineer this if we have to

In [63]:
# here, we just do a pivot table and just stick the artists in a couple of columns
# does not explode the columns of our base table
# https://stackoverflow.com/questions/22798934/pandas-long-to-wide-reshape
artists['idx'] = artists.groupby('reviewid').cumcount()
artists_final = artists.pivot(index='reviewid', columns='idx', values='artist')
artists_final.columns = ['artist1','artist2','artist3','artist4','artist5', 'artist6', 'artist7']
artists_final.reset_index(level=0, inplace=True)

In [64]:
artists_final.head()

,reviewid,artist1,artist2,artist3,artist4,artist5,artist6,artist7
0,1,aberfeldy,NaN,NaN,NaN,NaN,NaN,NaN
1,6,aarktica,NaN,NaN,NaN,NaN,NaN,NaN
2,7,aarktica,NaN,NaN,NaN,NaN,NaN,NaN
3,8,aarktica,NaN,NaN,NaN,NaN,NaN,NaN
4,10,abcs,NaN,NaN,NaN,NaN,NaN,NaN


# Part 7: Merging the clean datasets into one master table

In [65]:
# merge reviews and content
print(reviews_final.shape)
print(content_final.shape)
reviews_content = pd.merge(reviews_final, content_final, on = 'reviewid', how = 'left')
print(reviews_content.shape)

(18389, 13)
(18389, 2)
(18389, 14)


In [66]:
# merge in years
print(reviews_content.shape)
print(years_final.shape)
reviews_content_years = pd.merge(reviews_content, years_final, on = 'reviewid', how = 'left')
print(reviews_content_years.shape)

(18389, 14)
(17983, 3)
(18389, 16)


In [67]:
# merge in genres
print(genres_final.shape)
print(reviews_content_years.shape)
reviews_content_years_genres = pd.merge(reviews_content_years, genres_final, on = 'reviewid', how = 'left')
print(reviews_content_years_genres.shape)

(18389, 10)
(18389, 16)
(18389, 25)


In [68]:
# merge in labels
print(reviews_content_years_genres.shape)
print(labels_final.shape)
reviews_content_years_genres_lables = pd.merge(reviews_content_years_genres, labels_final, on = 'reviewid', how = 'left')
print(reviews_content_years_genres_lables.shape)

(18389, 25)
(18389, 6)
(18389, 30)


In [69]:
# merge in artists to produce the final dataset
print(reviews_content_years_genres.shape)
print(reviews_content_years_genres_lables.shape)
pitchfork_master = pd.merge(reviews_content_years_genres_lables, artists_final, on = 'reviewid', how = 'left')
print(pitchfork_master.shape)

(18389, 25)
(18389, 30)
(18389, 37)


In [70]:
# examine final dataset
pitchfork_master.head()

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,...,label3,label4,label5,artist1,artist2,artist3,artist4,artist5,artist6,artist7
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,...,NaN,NaN,NaN,massive attack,NaN,NaN,NaN,NaN,NaN,NaN
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,...,NaN,NaN,NaN,krallice,NaN,NaN,NaN,NaN,NaN,NaN
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,...,NaN,NaN,NaN,uranium club,NaN,NaN,NaN,NaN,NaN,NaN
3,22661,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,...,NaN,NaN,NaN,kleenex,liliput,NaN,NaN,NaN,NaN,NaN
4,22725,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,...,NaN,NaN,NaN,taso,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# where are there missing values?
# author type, eyar
pitchfork_master.isnull().sum()

reviewid              0
title                 0
artist                0
url                   0
score                 0
best_new_music        0
author                0
author_type        3904
pub_date              0
pub_weekday           0
pub_day               0
pub_month             0
pub_year              0
content               0
year1               406
year2             17670
electronic            0
experimental          0
folk/country          0
global                0
jazz                  0
metal                 0
pop/r&b               0
rap                   0
rock                  0
label1               38
label2            16761
label3            18227
label4            18380
label5            18387
artist1               0
artist2           17998
artist3           18351
artist4           18382
artist5           18385
artist6           18388
artist7           18388
dtype: int64